In this notebook, we 

In [227]:
import itertools

import dimod
import dimod.variables
import numpy as np
import pandas as pd

import sparse_qubo
import sparse_qubo.dwave

In [228]:
def create_scheduling_cost_matrix(
    num_row: int, num_col: int, max_cost: int = -1, min_cost: int = -5, seed: int | None = None
) -> np.ndarray:
    assert num_row >= 2
    assert num_col >= 2
    assert max_cost >= min_cost

    np.random.seed(seed)
    random_matrix = np.random.randint(min_cost, max_cost + 1, (num_row, num_col))

    return random_matrix.astype(float)


def create_incompatible_pairs(num_workers: int, num_combinations: int, seed: int | None = None) -> set[tuple[int, int]]:
    assert num_workers >= 2
    assert 1 <= num_combinations <= num_workers * (num_workers - 1) // 2

    np.random.seed(seed)
    all_pairs = list(itertools.combinations(range(num_workers), 2))
    np.random.shuffle(all_pairs)

    selected_pairs = set()
    usage_counts = np.zeros(num_workers, dtype=int)

    for _ in range(num_combinations):
        best_pair_idx = -1
        min_score = (float("inf"), float("inf"))
        for idx, (p1, p2) in enumerate(all_pairs):
            c1 = usage_counts[p1]
            c2 = usage_counts[p2]
            score = (max(c1, c2), c1 + c2)
            if score < min_score:
                min_score = score
                best_pair_idx = idx
            if score == (0, 0):
                break

        p1, p2 = all_pairs.pop(best_pair_idx)
        selected_pairs.add((p1, p2))
        usage_counts[p1] += 1
        usage_counts[p2] += 1

    return selected_pairs

In [229]:
def check_feasibility(solution_row: pd.Series, num_row: int, num_col: int, c_row: int, c_col: int) -> bool:
    for i in range(num_row):
        row_sum = sum(solution_row[f"X_{i}_{j}"] for j in range(num_col))
        if row_sum != c_row:
            return False
    for j in range(num_col):
        col_sum = sum(solution_row[f"X_{i}_{j}"] for i in range(num_row))
        if col_sum != c_col:
            return False
    return True


def check_incompatible_pairs(
    solution_row: pd.Series, num_row: int, num_col: int, incompatible_pairs: set[tuple[int, int]]
) -> bool:
    for i in range(num_row):
        workers_on_day = []
        for j in range(num_col):
            if solution_row[f"X_{i}_{j}"] == 1:
                workers_on_day.append(j)
        for p1, p2 in itertools.combinations(workers_on_day, 2):
            if (p1, p2) in incompatible_pairs or (p2, p1) in incompatible_pairs:
                return False
    return True

In [230]:
def create_scheduling_problem_bqm(
    network_type: sparse_qubo.NetworkType,
    num_row: int,
    num_col: int,
    c_row: int,
    c_col: int,
    incompatible_pairs: set[tuple[int, int]],
    cost_matrix: np.ndarray,
    penalty_factor: float = 1,
    seed=42,
):
    assert num_row >= 2
    assert num_col >= 2
    assert c_row >= 1
    assert c_col >= 1

    variables = dimod.variables.Variables([f"X_{i}_{j}" for i in range(num_row) for j in range(num_col)])
    bqm = dimod.BinaryQuadraticModel(dimod.BINARY)
    for row in range(num_row):
        row_vars = variables[row * num_col : (row + 1) * num_col]
        bqm += sparse_qubo.dwave.constraint(row_vars, sparse_qubo.ConstraintType.EQUAL_TO, network_type, c1=c_row)
    for col in range(num_col):
        col_vars = variables[col::num_col]
        bqm += sparse_qubo.dwave.constraint(col_vars, sparse_qubo.ConstraintType.EQUAL_TO, network_type, c1=c_col)

    for p1, p2 in incompatible_pairs:
        for row in range(num_row):
            bqm.add_quadratic(variables[row * num_col + p1], variables[row * num_col + p2], bias=1)

    bqm.scale(penalty_factor)

    for i in range(num_row):
        for j in range(num_col):
            bqm.add_linear(variables[i * num_col + j], cost_matrix[i, j])

    return bqm

In [231]:
num_days = 6
num_workers = 12
num_incompatible_pairs = 5
shifts_per_day = 2
shifts_per_worker = 1
naive_penalty_factor = 90
dc_penalty_factor = 15
seed = 42

In [232]:
incompatible_pairs = create_incompatible_pairs(num_workers, num_incompatible_pairs, seed=seed)
cost_matrix = create_scheduling_cost_matrix(num_days, num_workers, seed=seed)

print(incompatible_pairs)
print(cost_matrix)

{(0, 1), (3, 4), (2, 7), (6, 10), (8, 11)}
[[-2. -1. -3. -1. -1. -4. -3. -3. -3. -1. -2. -3.]
 [-1. -4. -2. -4. -2. -1. -5. -2. -4. -1. -2. -5.]
 [-5. -3. -3. -4. -2. -2. -3. -2. -2. -5. -3. -1.]
 [-3. -1. -5. -4. -2. -5. -2. -4. -4. -5. -4. -1.]
 [-4. -2. -2. -2. -2. -1. -3. -5. -2. -4. -2. -4.]
 [-4. -2. -1. -4. -4. -2. -4. -4. -2. -2. -5. -1.]]


In [233]:
naive_bqm = create_scheduling_problem_bqm(
    sparse_qubo.NetworkType.NAIVE,
    num_days,
    num_workers,
    shifts_per_day,
    shifts_per_worker,
    incompatible_pairs,
    cost_matrix,
    penalty_factor=naive_penalty_factor,
)
dc_bqm = create_scheduling_problem_bqm(
    sparse_qubo.NetworkType.DIVIDE_AND_CONQUER,
    num_days,
    num_workers,
    shifts_per_day,
    shifts_per_worker,
    incompatible_pairs,
    cost_matrix,
    penalty_factor=dc_penalty_factor,
)

In [234]:
n_inter = naive_bqm.num_interactions
dc_inter = dc_bqm.num_interactions
reduction = (1 - dc_inter / n_inter) * 100

print(f"{'Metric':<20} | {'Naive':<10} | {'D&C':<10}")
print("-" * 46)
print(f"{'Num Variables':<20} | {len(naive_bqm.variables):<10} | {len(dc_bqm.variables):<10}")
print(f"{'Num Interactions':<20} | {naive_bqm.num_interactions:<10} | {dc_bqm.num_interactions:<10}")
print(f"Interactions Reduced: {reduction:.1f}% ({n_inter} -> {dc_inter})")

Metric               | Naive      | D&C       
----------------------------------------------
Num Variables        | 72         | 720       
Num Interactions     | 576        | 558       
Interactions Reduced: 3.1% (576 -> 558)


In [235]:
import dwave.cloud
import dwave.system

In [236]:
client = dwave.cloud.Client.from_config()
print(client.get_solvers())

[BQMSolver(name='hybrid_binary_quadratic_model_version2p'), DQMSolver(name='hybrid_discrete_quadratic_model_version1p'), CQMSolver(name='hybrid_constrained_quadratic_model_version1p'), StructuredSolver(name='Advantage_system6.4', graph_id='01dae5a273'), NLSolver(name='hybrid_nonlinear_program_version1p'), StructuredSolver(name='Advantage_system4.1', graph_id='01d07086e1'), StructuredSolver(name='Advantage2_system1.11', graph_id='01dceba9f7')]


In [237]:
def sampling_with_dwave(
    solver_name: str,
    bqm: dimod.BinaryQuadraticModel,
    num_reads: int = 1000,
    annealing_time: float = 20,
    chain_strength: float | None = None,
):
    with dwave.system.EmbeddingComposite(dwave.system.DWaveSampler(solver=solver_name)) as sampler:
        result = sampler.sample(
            bqm,
            num_reads=num_reads,
            annealing_time=annealing_time,
            chain_strength=chain_strength,
            return_embedding=True,
        )

    return result

In [238]:
solver_name = "Advantage2_system1.11"
num_reads = 1000
annealing_time = 20

In [239]:
naive_result = sampling_with_dwave(
    solver_name,
    naive_bqm,
    num_reads=num_reads,
    annealing_time=annealing_time,
)

In [240]:
dc_result = sampling_with_dwave(
    solver_name,
    dc_bqm,
    num_reads=num_reads,
    annealing_time=annealing_time,
)

In [241]:
def calculate_chain_break_rate(df: pd.DataFrame) -> float:
    total_weighted_breaks = (df["chain_break_fraction"] * df["num_occurrences"]).sum()
    total_samples = df["num_occurrences"].sum()

    return total_weighted_breaks / total_samples

In [242]:
naive_embedding = sum([len(v) for v in naive_result.info["embedding_context"]["embedding"].values()])
dc_embedding = sum([len(v) for v in dc_result.info["embedding_context"]["embedding"].values()])

naive_first = naive_result.first.sample
dc_first = dc_result.first.sample

is_constrained_naive = check_feasibility(naive_first, num_days, num_workers, shifts_per_day, shifts_per_worker)
is_compatible_naive = check_incompatible_pairs(naive_first, num_days, num_workers, incompatible_pairs)
is_constrained_dc = check_feasibility(dc_first, num_days, num_workers, shifts_per_day, shifts_per_worker)
is_compatible_dc = check_incompatible_pairs(dc_first, num_days, num_workers, incompatible_pairs)

naive_df = naive_result.to_pandas_dataframe()
dc_df = dc_result.to_pandas_dataframe()

naive_energy_average = naive_df["energy"].mean()
dc_energy_average = dc_df["energy"].mean()


print(f"{'Metric':<20} | {'Naive':<10} | {'D&C':<10}")
print("-" * 46)
print(f"{'Feasible':<20} | {is_constrained_naive!s:<10} | {is_constrained_dc!s:<10}")
print(f"{'Compatible':<20} | {is_compatible_naive!s:<10} | {is_compatible_dc!s:<10}")
print(
    f"{'Chain Break Rate':<20} | {f'{calculate_chain_break_rate(naive_df):.3f}':<10} | {f'{calculate_chain_break_rate(dc_df):.3f}':<10}"
)
print(f"{'Best Solution Energy':<20} | {naive_result.first.energy:<10} | {dc_result.first.energy:<10}")
print(f"{'Average Energy':<20} | {f'{naive_energy_average:.3f}':<10} | {f'{dc_energy_average:.3f}':<10}")

Metric               | Naive      | D&C       
----------------------------------------------
Feasible             | True       | True      
Compatible           | True       | True      
Chain Break Rate     | 0.013      | 0.003     
Best Solution Energy | -34.0      | -45.0     
Average Energy       | 505.643    | 19.205    
